In [1]:
from source.source.postprocessing_utils import (
    get_uncertainty_scores,
    get_predicted_labels,
    get_missclassification_dataframe,
    get_ood_detection_dataframe,
    get_raw_scores_dataframe,
)

from source.datasets.constants import DatasetName
from source.losses.constants import LossName
from source.models.constants import ModelName
from source.metrics import (
    ApproximationType,
    GName,
    RiskType,
)

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)

stty: 'standard input': Inappropriate ioctl for device


In [2]:
base_score_dict = {
    "cross_entropy": "Logscore",
    # "brier_score": "Brier",
    # "spherical_score": "Spherical",
}

training_dataset_names = [
    "cifar10",
    # "cifar100",
    # "noisy_cifar100",
    # "missed_class_cifar10",
    # "noisy_cifar10",
]
temperature = 1.0
model_ids = np.arange(20)

list_extraction_datasets = [
    "cifar10",
    "cifar100",
    "svhn",
    "blurred_cifar100",
    "blurred_cifar10",
]
list_ood_datasets = [el for el in list_extraction_datasets]

loss_function_names = [el for el in LossName]

full_dataframe = None
full_ood_rocauc_dataframe = None
full_mis_rocauc_dataframe = None

In [ ]:
for training_dataset_name in training_dataset_names:
    if training_dataset_name not in [
        "missed_class_cifar10",
        "noisy_cifar10",
        "noisy_cifar100",
    ]:
        architectures = [ModelName.RESNET18, ModelName.VGG19]
        training_dataset_name_aux = training_dataset_name
    else:
        architectures = ["resnet18"]
        training_dataset_name_aux = training_dataset_name.split("_")[-1]
    for architecture in architectures:
        # try:
        uq_results, embeddings_per_dataset, targets_per_dataset = (
            get_uncertainty_scores(
                loss_function_names=loss_function_names,
                training_dataset_name=training_dataset_name,
                architecture=architecture,
                model_ids=model_ids,
                list_extraction_datasets=list_extraction_datasets,
                temperature=temperature,
                use_cached=False,
            )
        )

        df_ood = get_ood_detection_dataframe(
            ind_dataset=training_dataset_name_aux,
            uq_results=uq_results,
            list_ood_datasets=list_ood_datasets,
        )
        df_ood["architecture"] = architecture.value
        df_ood["training_dataset"] = training_dataset_name

        max_ind = int(
            targets_per_dataset[training_dataset_name_aux].shape[0] / len(model_ids)
        )
        true_labels = targets_per_dataset[training_dataset_name_aux][:max_ind]

        pred_labels = get_predicted_labels(
            embeddings_per_dataset=embeddings_per_dataset,
            training_dataset_name=training_dataset_name_aux,
        )

        df_misclassification = get_missclassification_dataframe(
            ind_dataset=training_dataset_name_aux,
            uq_results=uq_results,
            true_labels=true_labels,
            pred_labels=pred_labels,
        )
        df_misclassification["architecture"] = architecture.value
        df_misclassification["training_dataset"] = training_dataset_name

        # except Exception as ex:
        #     print(training_dataset_name, ex)
        #     continue

        scores_df_unravel = get_raw_scores_dataframe(uq_results=uq_results)
        scores_df_unravel["architecture"] = architecture.value
        scores_df_unravel["training_dataset"] = training_dataset_name

        if full_dataframe is None:
            full_dataframe = scores_df_unravel
            full_ood_rocauc_dataframe = df_ood
            full_mis_rocauc_dataframe = df_misclassification
        else:
            full_dataframe = pd.concat([full_dataframe, scores_df_unravel])
            full_ood_rocauc_dataframe = pd.concat([full_ood_rocauc_dataframe, df_ood])
            full_mis_rocauc_dataframe = pd.concat(
                [full_mis_rocauc_dataframe, df_misclassification]
            )

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [7]:
full_dataframe.sample(10)

,UQMetric,LossFunction,Dataset,Scores,architecture,training_dataset
738,BrierScore BayesRisk inner inner,CrossEntropy,blurred_cifar100,"[0.15089954, 0.35566884, 0.6705067, 0.51652044...",vgg19,cifar10
1471,SphericalScore ExcessRisk central central,CrossEntropy,cifar100,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",vgg19,cifar10
301,LogScore BayesRisk outer central,CrossEntropy,cifar100,"[0.0058252937, 0.22388887, 0.3887774, 0.747536...",vgg19,cifar10
1477,SphericalScore ExcessRisk central central,BrierScore,svhn,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",vgg19,cifar10
979,ZeroOneScore ExcessRisk outer central,CrossEntropy,blurred_cifar10,"[0.04507501, 0.032418795, 0.26323754, 0.046413...",vgg19,cifar10
196,LogScore ExcessRisk inner inner,CrossEntropy,cifar100,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",vgg19,cifar10
1555,SphericalScore BayesRisk inner inner,SphericalScore,cifar10,"[0.0616616, 0.06286067, 0.099069774, 0.1160330...",resnet18,cifar10
421,BrierScore TotalRisk outer inner,CrossEntropy,cifar100,"[0.0011294314, 0.41971236, 0.5720434, 0.788521...",vgg19,cifar10
677,BrierScore BayesRisk outer outer,CrossEntropy,svhn,"[0.19590339, 0.049960244, 0.2068328, 0.1087717...",vgg19,cifar10
504,BrierScore TotalRisk central outer,BrierScore,blurred_cifar10,"[0.14772677, 0.9475614, 0.9532074, 0.0642407, ...",resnet18,cifar10


In [ ]:
pattern_baserule = r'(Logscore|Brier|Neglog|Maxprob|Spherical)'
pattern_risk = r'(Total|Bayes|Excess|Reverse Bregman Information|Bregman Information|Expected Pairwise Bregman Information|MVBI|MV|BiasBI|Bias)'

full_ood_rocauc_dataframe['base_rule'] = full_ood_rocauc_dataframe['UQMetric'].str.extract(pattern_baserule)
full_ood_rocauc_dataframe['RiskType'] = full_ood_rocauc_dataframe['UQMetric'].str.extract(pattern_risk)
full_ood_rocauc_dataframe['LossFunction'] = full_ood_rocauc_dataframe['LossFunction'].replace(base_score_dict)

full_mis_rocauc_dataframe['base_rule'] = full_mis_rocauc_dataframe['UQMetric'].str.extract(pattern_baserule)
full_mis_rocauc_dataframe['RiskType'] = full_mis_rocauc_dataframe['UQMetric'].str.extract(pattern_risk)
full_mis_rocauc_dataframe['LossFunction'] = full_mis_rocauc_dataframe['LossFunction'].replace(base_score_dict)

full_dataframe['base_rule'] = full_dataframe['UQMetric'].str.extract(pattern_baserule)
full_dataframe['RiskType'] = full_dataframe['UQMetric'].str.extract(pattern_risk)
full_dataframe['LossFunction'] = full_dataframe['LossFunction'].replace(base_score_dict)

In [ ]:
full_dataframe.to_csv('../tables/new_tables/full_dataframe.csv')
full_ood_rocauc_dataframe.to_csv('../tables/new_tables/full_ood_rocauc.csv')
full_mis_rocauc_dataframe.to_csv('../tables/new_tables/full_mis_rocauc.csv')